In [ ]:
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
import cv2 as cv
from IPython.display import Image, clear_output

In [ ]:
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
%pip install -qr requirements.txt
import torch

Cloning into 'yolov7'...
remote: Enumerating objects: 1191, done.
remote: Total 1191 (delta 0), reused 0 (delta 0), pack-reused 1191
Receiving objects: 100% (1191/1191), 74.23 MiB | 18.18 MiB/s, done.
Resolving deltas: 100% (513/513), done.
/content/yolov7
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.8 MB/s eta 0:00:00


In [ ]:
%mkdir /content/yolov7/FI
%cd /content/yolov7/FI
!curl -L "https://universe.roboflow.com/ds/m6gxFOylAv?key=7QwGtYycDK" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: train/labels/naver_0204_jpg.rf.1bb63571e801bf905b04a9f1fccbc6e0.txt  
 extracting: train/labels/naver_0209_jpg.rf.c89a3ae165e61fcfa4458ba871d4dbd9.txt  
 extracting: train/labels/naver_0212_jpg.rf.a530f8def3bbd3f552f2028623548b8b.txt  
 extracting: train/labels/naver_0215_jpg.rf.9a84ac3f7e7f1a3fdc03aa1727c49f59.txt  
 extracting: train/labels/naver_0216_jpg.rf.3b26e62b1d372959637bca047f09f4a7.txt  
 extracting: train/labels/naver_0218_jpg.rf.f72e735fc6657e8544e9470aec8191c9.txt  
 extracting: train/labels/naver_0223_jpg.rf.43793f4e831b0b0ad58c6459fce64794.txt  
 extracting: train/labels/naver_0223_jpg.rf.7e986648645cfc0aefa60ce9455e1d36.txt  
 extracting: train/labels/naver_0224_jpg.rf.69213fe400bd60f4868a55d7bfc6d56f.txt  
 extracting: train/labels/naver_0227_jpg.rf.a0ff9eb520fdc369a64fef22284313fe.txt  
 extracting: train/labels/naver_0236_jpg.rf.a97cb0886093a08c1bf20392204bf503.txt  
 extracting: train/labels/naver_0245_jpg.rf.45f439a

In [ ]:
from glob import glob
train_img_list = glob('/content/yolov7/FI/train/images/*.jpg')
test_img_list = glob('/content/yolov7/FI/test/images/*.jpg')
valid_img_list = glob('/content/yolov7/FI/valid/images/*.jpg')
print(len(train_img_list), len(test_img_list), len(valid_img_list))

7065 1176 1660


In [ ]:
import yaml
with open('/content/yolov7/FI/train.txt','w') as f:
  f.write('₩n'.join(train_img_list)+'₩n')
with open('/content/yolov7/FI/test.txt','w') as f:
  f.write('₩n'.join(test_img_list)+'₩n')
with open('/content/yolov7/FI/valid.txt','w') as f:
  f.write('₩n'.join(valid_img_list)+'₩n')

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
  with open(line, 'w') as f:
    f.write(cell.format(**globals()))

In [ ]:
%cat /content/yolov7/FI/data.yaml

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 163
names: ['-', '10_pineapple', '10_tangerine', 'Apple', 'Bagel', 'Banana', 'Bitter melon', 'Cucumber', 'Garlic', 'Green Chili', 'Lady finger', 'Mushroom', 'Onion', 'Orange', 'Potato', 'Sponge Gourd', 'Tomato', 'ampalaya', 'apple', 'asparagus', 'avocado', 'bacon', 'banana', 'beef', 'bell pepper', 'bell_pepper', 'bento', 'bitter gourd', 'blueberries', 'bok choy', 'bottle', 'bread', 'broccoli', 'butter', 'cabbage', 'can', 'canned_tuna', 'carrot', 'cashew', 'cauliflower', 'cheese', 'cherry', 'chicken breast', 'chicken thigh', 'chicken wing', 'chicken', 'chicken_breast', 'chilli', 'chocolate', 'corn', 'crab', 'cream-cheese', 'cucumber', 'dates', 'egg', 'egg_', 'eggplant', 'eggs', 'fish', 'fish_cake', 'flour', 'french_fries', 'garlic', 'ginger', 'goat_cheese', 'grapefruit', 'grapes', 'grated_cheese', 'green chilli', 'green-chillies', 'green_beans', 'green_onion', 'ground_beef', 'guacamole', 'ham', 'hash_brown', 'heavy_cr

In [ ]:
with open('/content/yolov7/FI/data.yaml', 'w') as f:
    f.write('''
train: ./FI/train/images
test: ./FI/test/images
val: ./FI/valid/images

roboflow:
  workspace: dsstudy-h0rzy
  project: food-ingredients-image-detection_team4
  version: 1
  license: CC BY 4.0
  url: https://universe.roboflow.com/dsstudy-h0rzy/food-ingredients-image-detection_team4/dataset/1

nc: 163
names: ['-', '10_pineapple', '10_tangerine', 'Apple', 'Bagel', 'Banana', 'Bitter melon', 'Cucumber', 'Garlic', 'Green Chili', 'Lady finger', 'Mushroom', 'Onion', 'Orange', 'Potato', 'Sponge Gourd', 'Tomato', 'ampalaya', 'apple', 'asparagus', 'avocado', 'bacon', 'banana', 'beef', 'bell pepper', 'bell_pepper', 'bento', 'bitter gourd', 'blueberries', 'bok choy', 'bottle', 'bread', 'broccoli', 'butter', 'cabbage', 'can', 'canned_tuna', 'carrot', 'cashew', 'cauliflower', 'cheese', 'cherry', 'chicken breast', 'chicken thigh', 'chicken wing', 'chicken', 'chicken_breast', 'chilli', 'chocolate', 'corn', 'crab', 'cream-cheese', 'cucumber', 'dates', 'egg', 'egg_', 'eggplant', 'eggs', 'fish', 'fish_cake', 'flour', 'french_fries', 'garlic', 'ginger', 'goat_cheese', 'grapefruit', 'grapes', 'grated_cheese', 'green chilli', 'green-chillies', 'green_beans', 'green_onion', 'ground_beef', 'guacamole', 'ham', 'hash_brown', 'heavy_cream', 'humus', 'juice', 'ketchup', 'kimchi', 'kiwi', 'leek', 'lemon', 'lettuce', 'lime', 'lobster tails', 'mango', 'marmelade', 'mayonaise', 'mayonnaise', 'milk', 'mint', 'mozzarella cheese', 'mushrooms', 'mustard', 'nacho_chips', 'nuts', 'olives', 'onion', 'orange', 'oysters', 'pak_choi', 'parmasan_cheese', 'parsley', 'pasta', 'pawpaw', 'peach', 'peanuts', 'pear', 'peas', 'pepper', 'pickles', 'pimento', 'pineapple', 'plantains', 'plasticsaveholder', 'pomegrante', 'pork belly', 'pork', 'pot', 'potato', 'potatoes', 'pudding', 'pumpkin', 'radish', 'red chili', 'red_cabbage', 'red_grapes', 'red_onion', 'rice', 'rice_ball', 'rice_cake', 'salad', 'salami', 'salmon', 'sandwich', 'sausage', 'sayote', 'sea scallops', 'seaweed', 'seseme', 'shrimp', 'smoothie', 'spinach', 'spring_onion', 'strawberries', 'strawberry', 'sugar', 'sweet potato', 'sweet_potato', 'tangerine', 'tempeh', 'tofu', 'tomato', 'tomato_sauce', 'tortillas', 'tuna', 'turkey', 'watermelon', 'white rice', 'yogurt', 'zucchini']

''')

In [ ]:
%%time
%cd /content/yolov7/
!wandb disabled  # wandb를 비활성화합니다.
!python train.py --img 416 --batch 32 --epochs 50 --data ./FI/data.yaml --weights 'yolov7.pt'

/content/yolov7
/bin/bash: line 1: wandb: command not found
2023-09-25 09:25:21.899234: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-25 09:25:22.915213: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
YOLOR 🚀 v0.1-126-g84932d7 torch 2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16150.875MB)

Namespace(weights='yolov7.pt', cfg='', data='./FI/data.yaml', hyp='data/hyp.scratch.p5.yaml', epochs=50, batch_size=32, img_size=[416, 416], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=8, project='runs/train', enti

In [ ]:
! ls /content/yolov7/runs/train/evolve/

hyp.yaml  opt.yaml  weights


In [ ]:
with open('/content/yolov7/runs/train/evolve/results.txt', 'r') as file:
    content = file.read()
print(content)

FileNotFoundError: ignored

In [ ]:
Image(filename = '/content/yolov7/runs/train/FI_results/train_batch0.jpg',width = 1000)

In [ ]:
Image(filename = '/content/yolov7/runs/train/FI_results/val_batch0_labels.jpg',width = 1000)